In [ ]:
## AGENDA
## - Page Rank

In [ ]:
#from py2neo import Graph
#graph = Graph("bolt://neo4j.sktai.io:30074", auth=("neo4j","!neo4j00"))

In [ ]:
# https://github.com/neo4j-contrib/neo4j-apoc-procedures/releases/tag/3.5.0.1
#mv ./apoc-3.5.0.1-all.jar $Neo4J_HOME/plugins

In [1]:
from neo4j import GraphDatabase
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [2]:
#gph_conn = Neo4jConnection(uri="bolt://neo4j.sktai.io:30074", user="neo4j", pwd="!neo4j00")
#gph_conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="!neo4j00")
gph_conn = Neo4jConnection(uri="bolt://127.0.0.1:7687", user="neo4j", pwd="!neo4j00")

In [3]:
gph_conn.query("""
               CALL db.schema();
               """)

[<Record nodes=[<Node id=-5 labels=frozenset({'Node'}) properties={'indexes': [], 'name': 'Node', 'constraints': []}>] relationships=[<Relationship id=-5 nodes=(<Node id=-5 labels=frozenset({'Node'}) properties={'indexes': [], 'name': 'Node', 'constraints': []}>, <Node id=-5 labels=frozenset({'Node'}) properties={'indexes': [], 'name': 'Node', 'constraints': []}>) type='LINKED_TO' properties={}>]>]

In [4]:
gph_conn.query("""
          MATCH (n)
          DETACH DELETE n
          """)

[]

In [ ]:
## SET DATA

In [ ]:
G = {
        'A': {'B': 1, 'C': 1},
        'B': {'A': 1, 'C': 1},
        'C': {'A': 1, 'B': 1, 'D': 1},
        'D': {'C': 1, 'E': 1, 'G': 1},
        'E': {'D': 1, 'F': 1, 'G': 1},
        'F': {'E': 1, 'G': 1},
        'G': {'D': 1, 'E': 1, 'F': 1},
    }

In [ ]:
## Label Propagation from scratch

In [ ]:
from collections import Counter


def label_propagation(G, max_iterations=100):
    """Returns the community of the nodes in G
    based on the Label Propagation algorithm
    :param dict G: the graph
    :param int max_iter: max number of iterations
    """
    # initialize labels
    labels = {node:k  for k, node in enumerate(G)}
    print("======= Initialization")
    print(labels)
    
    # main loop
    for it in range(max_iterations):
        print("======= Iteration", it)
        old_labels = dict(labels)
        for node, neighbors in G.items():
            # counting the number of votes from each neighbors:
            votes = Counter([old_labels[n] for n in neighbors])
            print(node, votes)
            max_vote = -9999
            new_label = old_labels[node]
            for label, vote  in votes.items():
                if vote > max_vote:
                    max_vote = vote
                    new_label = label
                elif vote == max_vote:
                    if label > new_label:
                        new_label = label
            labels[node] = new_label            
        print(labels)
        # check convergence
        end = True
        for node in G:
            if old_labels[node] != labels[node]:
                end = False
                break
        if end:
            return labels
    raise Exception(
        f'Label propagation failed after max iteration = {max_iterations}'
    )

In [ ]:
## via GDS

In [5]:
gph_conn.query("""
               MERGE (A:Node {name: "A"})
               MERGE (B:Node {name: "B"})
               MERGE (C:Node {name: "C"})
               MERGE (D:Node {name: "D"})
               MERGE (E:Node {name: "E"})
               MERGE (F:Node {name: "F"})
               MERGE (G:Node {name: "G"})
               MERGE (Y:Node {name: "Y"})
               MERGE (Z:Node {name: "Z"})

               MERGE (A)-[:LINKED_TO]->(B)
               MERGE (B)-[:LINKED_TO]->(C)
               MERGE (C)-[:LINKED_TO]->(A)
               MERGE (D)-[:LINKED_TO]->(C)
               MERGE (D)-[:LINKED_TO]->(E)
               MERGE (E)-[:LINKED_TO]->(D)
               MERGE (E)-[:LINKED_TO]->(G)
               MERGE (F)-[:LINKED_TO]->(E)
               MERGE (G)-[:LINKED_TO]->(D)
               MERGE (G)-[:LINKED_TO]->(F)
               MERGE (Y)-[:LINKED_TO]->(Z)
               MERGE (Z)-[:LINKED_TO]->(Y)
               """)

[]

In [7]:
gph_conn.query("""
               CALL gds.graph.drop("graph")
               """)
gph_conn.query("""
               CALL gds.graph.create("graph", "Node", {LINKED_TO: {type: 'LINKED_TO', orientation: 'UNDIRECTED'}})
               """)

[<Record graphName='graph' nodeProjection={'Node': {'properties': {}, 'label': 'Node'}} relationshipProjection={'LINKED_TO': {'orientation': 'UNDIRECTED', 'aggregation': 'DEFAULT', 'type': 'LINKED_TO', 'properties': {}}} nodeCount=9 relationshipCount=24 createMillis=3>]

In [8]:
gph_conn.query("""
               CALL gds.labelPropagation.stream("graph")
               YIELD nodeId, communityId
               RETURN gds.util.asNode(nodeId).name AS nodeName, communityId
               ORDER BY communityId
               """)

[<Record nodeName='A' communityId=1>,
 <Record nodeName='B' communityId=1>,
 <Record nodeName='C' communityId=1>,
 <Record nodeName='D' communityId=30>,
 <Record nodeName='E' communityId=30>,
 <Record nodeName='F' communityId=30>,
 <Record nodeName='G' communityId=30>,
 <Record nodeName='Y' communityId=34>,
 <Record nodeName='Z' communityId=34>]

In [12]:
## Prior Knowledge on Membership
gph_conn.query("""
               MATCH (A:Node {name: "A"}) SET A.knownCommunity = 0
               """)
gph_conn.query("""
               MATCH (B:Node {name: "B"}) SET B.knownCommunity = 0
               """)
gph_conn.query("""
               MATCH (F:Node {name: "F"}) SET F.knownCommunity = 1
               """)
gph_conn.query("""
               MATCH (G:Node {name: "G"}) SET G.knownCommunity = 1
               """)

[]

In [14]:
gph_conn.query("""
               CALL gds.graph.drop("graph")
               """)
gph_conn.query("""
               CALL gds.graph.create("graph", {Node: {label: "Node", properties: "knownCommunity"}}, 
                                              {LINKED_TO: {type: 'LINKED_TO', orientation: 'UNDIRECTED'}})
               """)

Query failed: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `gds.graph.drop`: Caused by: java.lang.IllegalArgumentException: Graph with name `graph` does not exist and can't be removed.}


[<Record graphName='graph' nodeProjection={'Node': {'properties': {'knownCommunity': {'property': 'knownCommunity', 'defaultValue': nan}}, 'label': 'Node'}} relationshipProjection={'LINKED_TO': {'orientation': 'UNDIRECTED', 'aggregation': 'DEFAULT', 'type': 'LINKED_TO', 'properties': {}}} nodeCount=9 relationshipCount=24 createMillis=16>]

In [15]:
gph_conn.query("""
               CALL gds.labelPropagation.stream("graph", {seedProperty: "knownCommunity"})
               YIELD nodeId, communityId
               RETURN gds.util.asNode(nodeId).name AS nodeName, communityId
               ORDER BY communityId
               """)

[<Record nodeName='A' communityId=0>,
 <Record nodeName='B' communityId=0>,
 <Record nodeName='C' communityId=0>,
 <Record nodeName='D' communityId=1>,
 <Record nodeName='E' communityId=1>,
 <Record nodeName='F' communityId=1>,
 <Record nodeName='G' communityId=1>,
 <Record nodeName='Y' communityId=36>,
 <Record nodeName='Z' communityId=36>]